In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../../data/openings.csv')

In [3]:
df

,id,project_id,title,description,tags
0,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
1,aa0d5847-b00b-42e0-8986-6185ecffd449,22a9acc5-428a-41d4-b056-9bebe9183afe,Content Writer,Some description,[]
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,22a9acc5-428a-41d4-b056-9bebe9183afe,Manager,management,[]
3,1067c159-be01-4f89-a1cf-a32cdd67bb72,6f6235eb-a751-43f2-8bad-506c884635db,Backend Developer,This is a new testing description,"['golang', 'javascript', 'django']"
4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,6f6235eb-a751-43f2-8bad-506c884635db,Frontend Developer,need a good and fast paced frontend developer ...,"['react', 'next', 'ts']"


In [4]:
from keras.models import load_model
from tensorflow import newaxis
import json

model = load_model('../../../models/projects/recommendations.h5')

with open('../../../models/projects/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/projects/project_embeddings.json') as f:
    project_embeddings = json.load(f)
with open('../../../models/projects/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/projects/project_bias_embeddings.json') as f:
    project_bias_embeddings = json.load(f)

def predict_score(user_id, project_id):
    user_embedding = np.array(user_embeddings[user_id])
    project_embedding = np.array(project_embeddings[project_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, project_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    project_bias_embedding = np.array(project_bias_embeddings[project_id])

    user_bias_embedding = user_bias_embedding[:, newaxis]
    project_bias_embedding = project_bias_embedding[:, newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, project_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]
    return predicted_rating.numpy()

def recommend(userID):
    df = pd.read_csv('../../../data/project_scores.csv')
    user_ratings = df[df['user_id'] == userID]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    recommendation = df[~df['project_id'].isin(user_ratings['project_id'])][['project_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict_score(userID, x['project_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec['project_id'].values.tolist()

In [5]:
recommended_projects=recommend('5f5a90e2-6601-480c-bc08-7592e06beacd')

In [6]:
recommended_openings = df[df['project_id'].isin(recommended_projects)]

In [7]:
recommended_openings

,id,project_id,title,description,tags
0,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
1,aa0d5847-b00b-42e0-8986-6185ecffd449,22a9acc5-428a-41d4-b056-9bebe9183afe,Content Writer,Some description,[]
2,c888d326-0c57-4ac5-8629-5ee397fd0f6c,22a9acc5-428a-41d4-b056-9bebe9183afe,Manager,management,[]


In [8]:
with open('../../../models/openings/user_embeddings.json') as f:
    user_embeddings = json.load(f)
with open('../../../models/openings/opening_embeddings.json') as f:
    opening_embeddings = json.load(f)
with open('../../../models/openings/user_bias_embeddings.json') as f:
    user_bias_embeddings = json.load(f)
with open('../../../models/openings/opening_bias_embeddings.json') as f:
    opening_bias_embeddings = json.load(f)

In [9]:
import tensorflow as tf

model = load_model('../../../models/openings/recommendations.h5')

def predict(user_id, opening_id):
    user_embedding = np.array(user_embeddings[user_id])
    opening_embedding = np.array(opening_embeddings[opening_id])

    # Passing user embedding and movie embedding through the concat layer
    concatenated_embeddings = model.get_layer('concat')([user_embedding, opening_embedding])

    # Passing the concatenated embeddings through the dense layers
    x = model.get_layer('dense1')(concatenated_embeddings)
    x = model.get_layer('dense2')(x)

    user_bias_embedding = np.array(user_bias_embeddings[user_id])
    opening_bias_embedding = np.array(opening_bias_embeddings[opening_id])

    user_bias_embedding = user_bias_embedding[:, tf.newaxis]
    opening_bias_embedding = opening_bias_embedding[:, tf.newaxis]

    # Combine embeddings, biases, and pass through the output layer
    input_tensors = [x, user_bias_embedding, opening_bias_embedding]
    concatenated_features = model.get_layer('combined_features')(input_tensors)
    
    x = model.get_layer('combined_dense1')(concatenated_features)
    x = model.get_layer('combined_dense2')(x)
    
    x = model.get_layer('output')(x)

    predicted_rating = x[0][0]

    return predicted_rating.numpy()

In [10]:
scores = pd.read_csv('../../../data/opening_scores.csv')
scores = scores.merge(df, left_on='opening_id', right_on='id',)

def user_recommendations(user_id):
    user_ratings = scores[scores['user_id'] == user_id]
    user_ratings = user_ratings[user_ratings['score'] != 1]
    
    recommended_projects=recommend(user_id)
    
    recommendation = scores[~scores['opening_id'].isin(user_ratings['opening_id'])].drop_duplicates()
    recommendation = recommendation[recommendation['project_id'].isin(recommended_projects)][['opening_id']].drop_duplicates()
    recommendation['score_predict'] = recommendation.apply(lambda x: predict(user_id, x['opening_id']), axis=1)
    
    final_rec = recommendation.sort_values(by='score_predict', ascending=False)
    return final_rec

In [11]:
user_recommendations('5f5a90e2-6601-480c-bc08-7592e06beacd')

,opening_id,score_predict
0,d569ec70-f3fa-421c-88db-0773b26a2475,3.541359


In [12]:
scores

,user_id,opening_id,score,id,project_id,title,description,tags
0,0db6b893-14aa-4ae5-8e57-692ca7943ab5,d569ec70-f3fa-421c-88db-0773b26a2475,4,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
1,90bab470-e5d4-4d93-bd0a-15c51918f172,d569ec70-f3fa-421c-88db-0773b26a2475,4,d569ec70-f3fa-421c-88db-0773b26a2475,22a9acc5-428a-41d4-b056-9bebe9183afe,Designer,Need a good designer fast,"['figma', 'ui', 'ux']"
2,0db6b893-14aa-4ae5-8e57-692ca7943ab5,1067c159-be01-4f89-a1cf-a32cdd67bb72,4,1067c159-be01-4f89-a1cf-a32cdd67bb72,6f6235eb-a751-43f2-8bad-506c884635db,Backend Developer,This is a new testing description,"['golang', 'javascript', 'django']"
3,90bab470-e5d4-4d93-bd0a-15c51918f172,1067c159-be01-4f89-a1cf-a32cdd67bb72,4,1067c159-be01-4f89-a1cf-a32cdd67bb72,6f6235eb-a751-43f2-8bad-506c884635db,Backend Developer,This is a new testing description,"['golang', 'javascript', 'django']"
4,90bab470-e5d4-4d93-bd0a-15c51918f172,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,6f6235eb-a751-43f2-8bad-506c884635db,Frontend Developer,need a good and fast paced frontend developer ...,"['react', 'next', 'ts']"
5,5f5a90e2-6601-480c-bc08-7592e06beacd,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,4,2bc22ae6-ac3d-4aa5-82a4-dd55b8d8709b,6f6235eb-a751-43f2-8bad-506c884635db,Frontend Developer,need a good and fast paced frontend developer ...,"['react', 'next', 'ts']"
